## Secret Sharing ##

### Task 1: McEliece-Sarwate Secret Sharing ###

Let us consider a setting in which you need to remember a passphrase $\mathbf{m} = (m_1,m_2,m_3,m_4) \in \mathbb{F}^4$. The size of each message symbol is $8$ bit. As you do not want to learn your passphrase by heart, you think of simply storing it on a USB drive. However, it may happen that you lose the drive. Therefore, you decide to employ a McEliece-Sarwate secret sharing scheme and distribute the information about your passphrase to $6$ distinct USB drives. If you lose one of the drives, a person who finds it should not be able to reconstruct your passphrase.

- Which values $\left( n,k,z \right)$ would you use for the secret sharing scheme? Add the values in the code.
- Which finite field $\mathbb{F}$ is best suited to represent the message symbols? Choose a value for $\tt{q}$ and represent the values in $\tt{password\_unconverted}$ as elements in $\mathbb{F}$. Store the result in $\tt{password}$.

In [1]:
# Choose your parameters here:
q = 2**61 -1 # your code here
z = 1# your code here
k = 5# your code here
n = 6# your code here

F.<x> = GF(q)

# choose your password (= the message) here or keep the one provided
password_unconverted = [9, 5, 14, 6]
password = password_unconverted # your code here

print("=== Chosen passphrase: ===")
print(password)

=== Chosen passphrase: ===
[9, 5, 14, 6]


- Implement the function $\tt{generate\_vandermonde(F, n, k)}$. Generate a Vandermonde matrix of size $n \times k$ in $\mathbb{F}$.

In [2]:
def generate_vandermonde(F, n, k):
    """
    Generate a n x k Vandermonde matrix
    Arguments:
        F : Finite Field
        n, k: int
    Returns:
        matrix : Vandermonde matrix
    """
    pass # your code here
    G_1 = sage.matrix.special.vandermonde([Integer(i) for i in range(1,n+1)],ring=F)
    G = G_1[:,:k]
    
    return G

In [3]:
G = generate_vandermonde(F, n, k)
print("=== Vandermonde matrix ===")
print(G)

=== Vandermonde matrix ===
[   1    1    1    1    1]
[   1    2    4    8   16]
[   1    3    9   27   81]
[   1    4   16   64  256]
[   1    5   25  125  625]
[   1    6   36  216 1296]


- Implement the encoding function $\tt{encode\_vandermonde(F, n, k, z, m, G)}$. How many random symbols have to be generated? Combine the random symbols and the message symbol in a coefficient vector $\tt{c}$.

In [4]:
import random

def encode_vandermonde(F, n, k, z, m, G):
    """
    Encode the message of length k-z into a vector of n codeword symbols using
    a Vandermonde matrix
    Arguments:
        F : Finite field
        n, k, z : int
        m : list containing message symbols
        G : Vandermonde matrix
    Returns:
        list: A list containing the encoded symbols (shares)
    """
    # Generate coefficients vector from z random elements and k-z message symbols
    c = [randint(0,q-1) for i in range(z)] + m# your code here
    c = vector(F,c)

    print("=== Coefficient vector ===")
    print(c)

    shares = G * c# your code here
    
    assert len(shares) == n, "The number of shares is not correct"

    return shares

In [5]:
# Encode the shares
shares0 = encode_vandermonde(F, n, k, z, password, G)

print("=== Encoding done! Stored shares: ===")
print(shares0)

=== Coefficient vector ===
(110347058327746319, 9, 5, 14, 6)
=== Encoding done! Stored shares: ===
(110347058327746353, 110347058327746565, 110347058327747255, 110347058327748867, 110347058327751989, 110347058327757353)


- Now we will simulate one (or more) of the devices getting lost: You receive all but $\tt{num\_erasures}$ of the shares and a list $\tt{erasures}$ of the drives that were lost

In [6]:
# Simply run this box, it removes the shares of one or more of the devices
num_erasures = 1 # number of drives that gets lost

import random
erasures = random.sample(range(0, n), num_erasures)
print("Drive", erasures, "was lost!")

received_shares = [shares0[i] for i in range(n) if i not in erasures]

print("\n")
print("=== Received shares: ===")
print(received_shares)

Drive [0] was lost!


=== Received shares: ===
[110347058327746565, 110347058327747255, 110347058327748867, 110347058327751989, 110347058327757353]


- Implement the decoding function $\tt{decode\_vandermonde(F, k, z, shares, e, G)}$, which shall reconstruct the coefficient vector $\tt{c}$ from the given shares. Extract the message symbols from $\tt{c}$ to obtain the message $\tt{m}$.
*Hint*: At first, construct an effective Vandermonde matrix which represents only the rows related to the given shares. The indices of erased shares are given as a list $\tt{erasures}$.

In [7]:
def decode_vandermonde(F, k, z, shares, e, G):
    """
    Reconstruct the message symbols from k code symbols by inversion of the Vandermonde matrix
    Arguments:
        F : Finite field
        k, z : int
        shares : list containing the shares
        e : list, Positions of erasures
        G : Encoding Vandermonde matrix
    Returns:
        list : A list containing the decoded message symbols
    """

    pass # your code here
    zeilen_behalten = [i for i in range(n) if i not in e]
    G_gekuerzt = G[zeilen_behalten]
    inv_G = G_gekuerzt.inverse()
    mv = inv_G * vector(F,shares)
    m = list(mv)[z:]
    return m  # make sure m is a list and not a vector

In [8]:
# Reconstruct the password from the retained drives
password_reconstructed = decode_vandermonde(F, k, z, received_shares, erasures, G)
print("=== Reconstructed password: ===")
print(password_reconstructed)

assert password_reconstructed == password, 'Password was not restored correctly.'
print('Correct!')

=== Reconstructed password: ===
[9, 5, 14, 6]
Correct!


#### Key Notes for Task 1: ####
- Explain your choice of parameters n, k, z and what meaning they have in the context of our problem. Were other choices possible?
  
N and z were both given in the task, I could not change them. They mean that we split our password in $n=6$ shares and if $z=1$ shares get leaked, no information can be retrieved with only $z=1$ shares. Since we want to protect 4 message symbols, $k-z=k-1=4$ -> $k=5$
  
- Could we also have used Shamir's secret sharing in this setting? If yes, how would the parameters differ?

Shamir's Secret scheme protects one symbol of information. We need to protect 4, so no, we cannot use SSSS here.

- So far, we've only considered secret sharings in which all parties are honest. If one of the devices were corrupted, i.e., sending the wrong shares when queried, would we retain our (n,k,z)-secret sharing? If no, which parameter changes? Can you as the dealer figure out which device is corrupted?

If I get shares to reconstruct the secret and one share is garbage, the overall result will be garbage, the secret cannot be reconstructed anymore.

### Task 2: Threshold-Changeable Secret Sharing

As discussed in the tutorial, it is possible to create secret sharing schemes that can change the recovery threshold $k$ and the security threshold $z$ by converting the user's shares locally. **Folding** is one method how to construct such a threshold-changeable secret sharing scheme. In this task, you are asked to implement such a scheme. The secret sharing scheme shall encode the message $\mathbf{m}$, which is already given in the code template.

- First, create a $(18,12,6)$ McEliece-Sarwate secret sharing using your implementation from Task 1. Store the shares in the list $\tt{shares0}$.

In [9]:
# Parameter definitions
n0 = 18
k0 = 12
z0 = 6
F = GF(q)

m = [F(1), F(2), F(3), F(4), F(5), F(6)]
print("=== Secret message: ===")
print(m)

# 2.1: Generation of the initial secret sharing: Encode m into n0 shares using your implementation from part 1

G = generate_vandermonde(F, n0, k0)
shares0 = encode_vandermonde(F, n0, k0, z0, m, G)# your code here

print("=== Initial shares: ===")
print(shares0)

=== Secret message: ===
[1, 2, 3, 4, 5, 6]
=== Coefficient vector ===
(903091802757025808, 1973864888687788196, 1706703474078970559, 1854761414509883850, 964843254119694890, 1649162490795147289, 1, 2, 3, 4, 5, 6)
=== Initial shares: ===
(2134898297307428760, 186855186125955277, 65885572142674050, 1459034752701960858, 636313332023097893, 1576912371381435929, 1731145440897600310, 840820771546340608, 536241389146330077, 1627362278167518676, 1394947619476410732, 105100195455179687, 606261985151813152, 1008686078257359308, 1198967174756869333, 131200833304634265, 35672878300364998, 1404178359105220634)


- Create $n=6$ shares by folding $v=3$ of the original shares into lists of size $3$, respectively. Implement the function $\tt{fold(F,v,shares)}$. Store the new shares in the list $\tt{shares1}$ (which is then a list of lists, **do not concatenate**). The new shares establish a $(6,4,2)$ secret sharing scheme.

In [10]:
v = 3

def fold(F, v, shares):
    """
    Fold v shares from a list of n shares into a new list of n/v shares.
    Arguments:
        F : Finite field
        v : int
        shares : list of shares to fold
    Returns:
        new_shares : A list of new shares, each of which is a list of size v
    """
    assert (len(shares) % v) == 0, "The number v of shares to fold must devide the total number of shares"
    
    new_shares = []
    anzahl_paare = ceil(len(shares)/v)
    for i in range(anzahl_paare):
        new_shares.append(list())

    #print(len(shares))
    for i in range(len(shares)):
        ganzzahl_div = i // v
        #print(ganzzahl_div)
        new_shares[ganzzahl_div].append(shares[i])

    # your code here
    
    return new_shares

print("Folding shares...")
shares1 = fold(F, v, shares0)
print("=== New shares: ===")
print(shares1)

Folding shares...
=== New shares: ===
[[2134898297307428760, 186855186125955277, 65885572142674050], [1459034752701960858, 636313332023097893, 1576912371381435929], [1731145440897600310, 840820771546340608, 536241389146330077], [1627362278167518676, 1394947619476410732, 105100195455179687], [606261985151813152, 1008686078257359308, 1198967174756869333], [131200833304634265, 35672878300364998, 1404178359105220634]]


- Complete the implementation of the function $\tt{collect\_and\_decode()}$. It should read the devices sequentially, and try to decode the secret message from as few devices as possible. How many devices does one have to read until successful decoding of the message?

In [11]:
def collect_and_decode(F, k0, z0, v, devices, G):
    """
    Collect shares until decoding is possible. Raises an exception if decoding fails completely.
    Arguments:
        F : Finite field
        k0, z0, v : int
        devices : A list of lists containing the devices' shares
        G : Encoding Vandermonde matrix
    Returns:
        secret: A list containing the decoded message symbols
        devices_needed: An integer representing the number of devices that were read
    """
    print("Collecting shares:\n")
    decodable = False
    for i, d in enumerate(devices): 
        
        print("Device no.", i+1)
        pass # your code here, try to decode with only the devices read so far (first with one, then with two, three, ..., until decoding is possible)
        if v * (i+1) >= k0:
            received_shares = [
                x
                for xs in devices[:(i+1)]
                for x in xs
            ]
            decodable = True
            erasures = [i for i in range(len(received_shares),len(devices)*v)]
            if v == 2:
                erasures = [i for i in range(2,(v+1)*len(devices),3)]
                #print(erasures)
                #return
                
            #print(erasures)
            break
    if decodable:
        #print(k0)
        #print(z0)
        #print(received_shares)
        #print(erasures)
        #print(G)
        password_reconstructed = decode_vandermonde(F, k0, z0, received_shares, erasures, G)
        return password_reconstructed, (i+1)
        
    raise Exception("Could not decode from the given %d shares" % len(devices))
    
n = n0
m_reconstructed, num_devices = collect_and_decode(F, k0, z0, v, shares1, G)
assert m == m_reconstructed, "message was not correctly reconstructed"
print('message successfully decoded from', num_devices, 'devices!')


Device no. 1
Device no. 2
Device no. 3
Device no. 4
message successfully decoded from 4 devices!


- Complete the function $\tt{share\_conversion()}$ which simply removes the last element of a local share. Use this function on every share in the $\tt{shares1}$ list. Run $\tt{collect\_and\_decode()}$ again (make sure that it handles the different length of the share vectors correctly!). How many devices does one have to read now?

In [12]:
# 2.4: Share conversion
def share_conversion(share):
    shorter_share = [share[i] for i in range(len(share)-1)]# your code here
    return shorter_share

# apply to each share in shares1
shortened_shares = [share_conversion(share) for share in shares1]# your code here
print(shortened_shares)
print("\n")
print("=== Collecting shares and trying to decode after share conversion: ===")
m_reconstructed, num_devices = collect_and_decode(F, k0, z0, v-1, shortened_shares, G)

assert m == m_reconstructed, "message was not correctly reconstructed"
print('message successfully decoded from', num_devices, 'devices!')

[[2134898297307428760, 186855186125955277], [1459034752701960858, 636313332023097893], [1731145440897600310, 840820771546340608], [1627362278167518676, 1394947619476410732], [606261985151813152, 1008686078257359308], [131200833304634265, 35672878300364998]]


=== Collecting shares and trying to decode after share conversion: ===

Device no. 1
Device no. 2
Device no. 3
Device no. 4
Device no. 5
Device no. 6
message successfully decoded from 6 devices!


#### Key Notes for Task 2: ####
- in the final subtask, we deleted the last element of each device's share. Does the scheme still work if we delete a different element from each device in the 'folded' secret sharing?

It does if the decode function knows exactly which shares have been removed. If the decoding schemes assumes we deleted the last share from all folded shares and we instead did delete something else, it is broken.
- What are the parameters of our final secret sharing after the removal of one element from each share in $\tt{shares1}$?

$n=6$ because we have (originally) pairs of $3$ shares. If we then discard one share from every share-triple, $k=6$ because $k_0=12$ and every former triple now contains $2$ shares. $z_0 = 6$, after putting shares into pairs of $2$, the new $z=3$.